## Open Quantum System Simulation: Single Atom in a Cavity

In this simulation, we study the dynamics of a **single two-level atom** coupled to a **quantized cavity mode**.  
At this stage, the **atom–cavity coupling strength is set to zero (`g = 0`)**, so the atom and cavity do not exchange excitations.  
Instead, the evolution is governed purely by **dissipative processes**:

- **Cavity photon leakage (γ)**  
- **Atomic spontaneous decay (κ)**  
- **Atomic pure dephasing (β)**  

We start with the cavity in the **single-photon state** `|1⟩` and the atom in the **ground state** `|g⟩`.  
The interactive sliders below allow you to explore how the different dissipation mechanisms affect the system’s observables:

- ⟨n_atom⟩ – atomic excitation probability  
- ⟨n_cavity⟩ – photon number in the cavity  
- |coherence| – atomic coherence (off-diagonal terms of the density matrix)  


In [8]:
import numpy as np
import qutip as qt
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider
from system_single import AtomCavitySystem
from system_multiple import MultipleAtomsCavitySystem


In [9]:
def run_simulation(gamma, kappa, beta):
    # Create system
    system = AtomCavitySystem(N=10, g=0, w0=(1.0 * 2 * np.pi),
                              gamma=gamma, kappa=kappa, beta=beta)

    # Build Hamiltonian
    H = system.create_hamiltonian()

    # Initial state |0⟩_cavity ⊗ |+⟩_atom
    psi11 = system.create_initial_state('1', 'g')

    # Collapse operators
    c_ops = system.create_collapse_operators(leaking=True, decay=True, dephasing=True)

    # Expectation values
    e_ops = system.create_expectation_values(
        number_atom=True,
        number_cavity=True,
        coherence_ge=True,
        coherence_eg=True
    )

    # Time evolution
    times = np.linspace(0, 20, 500)
    result = qt.mesolve(H, psi11, times, c_ops, e_ops)

    # Plot
    plt.figure(figsize=(8, 5))
    plt.plot(times, result.expect[0], label="⟨n_atom⟩")
    plt.plot(times, result.expect[1], label="⟨n_cavity⟩")
    plt.plot(times, np.abs(result.expect[2]), label="|coherence (atom)|")
    plt.legend()
    plt.xlabel("Time")
    plt.ylabel("Expectation value")
    plt.grid(True)
    plt.show()

# Create interactive sliders
interact(run_simulation,
         gamma=FloatSlider(min=0, max=0.5, step=0.01, value=0, description="Leaking γ"),
         kappa=FloatSlider(min=0, max=0.5, step=0.01, value=0, description="Decay κ"),
         beta=FloatSlider(min=0, max=0.5, step=0.01, value=0, description="Dephasing β"));


interactive(children=(FloatSlider(value=0.0, description='Leaking γ', max=0.5, step=0.01), FloatSlider(value=0…

## Introducing Atom–Cavity Coupling

Now we extend the setup by introducing a **finite atom–cavity coupling strength**  
\[
g = 0.1 \cdot 2\pi
\]  
between the single two-level atom and the quantized cavity mode.  

This coupling allows for **coherent exchange of excitations** between the atom and the cavity, leading to **Rabi oscillations** in the absence of dissipation.  

The initial state is the same as before:  
- The cavity is prepared in the **single-photon state** `|1⟩`.  
- The atom starts in the **ground state** `|g⟩`.  

Dissipation mechanisms are again controlled by the sliders:  
- **Photon leakage (γ)** from the cavity  
- **Atomic decay (κ)** into the environment  
- **Atomic dephasing (β)**  

By turning on the coupling, we can now observe how the **competition between coherent dynamics and dissipation** shapes the evolution of:  
- ⟨n_atom⟩ – the atomic excitation probability  
- ⟨n_cavity⟩ – the cavity photon number  
- |coherence| – the atomic coherence  (which is still zero due to the chosen initial state)


In [10]:
def run_simulation(gamma, kappa, beta):
    # Create system
    system = AtomCavitySystem(N=10, g=(0.1 * 2 * np.pi), w0=(1.0 * 2 * np.pi),
                              gamma=gamma, kappa=kappa, beta=beta)

    # Build Hamiltonian
    H = system.create_hamiltonian()

    # Initial state |0⟩_cavity ⊗ |+⟩_atom
    psi11 = system.create_initial_state('1', 'g')

    # Collapse operators
    c_ops = system.create_collapse_operators(leaking=True, decay=True, dephasing=True)

    # Expectation values
    e_ops = system.create_expectation_values(
        number_atom=True,
        number_cavity=True,
        coherence_ge=True,
        coherence_eg=True
    )

    # Time evolution
    times = np.linspace(0, 20, 500)
    result = qt.mesolve(H, psi11, times, c_ops, e_ops)

    # Plot
    plt.figure(figsize=(8, 5))
    plt.plot(times, result.expect[0], label="⟨n_atom⟩")
    plt.plot(times, result.expect[1], label="⟨n_cavity⟩")
    plt.plot(times, np.abs(result.expect[2]), label="|coherence (atom)|")
    plt.legend()
    plt.xlabel("Time")
    plt.ylabel("Expectation value")
    plt.grid(True)
    plt.show()

# Create interactive sliders
interact(run_simulation,
         gamma=FloatSlider(min=0, max=0.5, step=0.01, value=0, description="Leaking γ"),
         kappa=FloatSlider(min=0, max=0.5, step=0.01, value=0, description="Decay κ"),
         beta=FloatSlider(min=0, max=0.5, step=0.01, value=0, description="Dephasing β"));


interactive(children=(FloatSlider(value=0.0, description='Leaking γ', max=0.5, step=0.01), FloatSlider(value=0…

## Spin Echo with Quasi-Static Noise

Now we study the effect of **quasi-static dephasing noise** on a single two-level atom in a cavity.  

### Key differences from the previous setups:
- The **atom–cavity coupling is set to zero** (`g = 0`), so there is no coherent exchange of excitations.  
- The atom is initialized in the **superposition state** `|+⟩ = (|g⟩ + |e⟩)/√2`, which carries **coherence** between ground and excited states.  
- The noise is modeled as a **static frequency detuning** of the atom, implemented as a Hamiltonian term proportional to **σ_z**.  
  - In physical terms, this corresponds to a random offset of the atomic transition frequency.  
  - On the Bloch sphere, it is equivalent to a **rotation around the z-axis**, leading to phase drift of the superposition.  
- Dissipation is still possible via **photon leakage (γ)** and **atomic decay (κ)**, but we deliberately omit Lindblad pure dephasing.  

### Spin echo protocol:
At a chosen **echo time** `t_echo`, we apply a **π-pulse** (a rotation around the X-axis).  
This reverses the relative phase accumulation caused by the static noise, so that the atomic coherence partially **refocuses** at later times.  

### What we observe:
- **⟨n_atom⟩** – atomic excitation probability  
- **⟨n_cavity⟩** – cavity photon number  
- **|coherence|** – the magnitude of atomic coherence  

Without the echo, quasi-static noise would cause an irreversible decay of coherence.  
By inserting the π-pulse, we see the **coherence increase again after `t_echo`**, demonstrating the recovery effect of the spin echo.  


In [ ]:
def run_simulation(gamma, kappa, sigma, t_echo):
    num_realizations=20
    # Create system (no Lindblad dephasing here, only leakage/decay)
    system = AtomCavitySystem(N=10, g=0, w0=(1.0 * 2 * np.pi),
                              gamma=gamma, kappa=kappa, beta=0.0)

    H = system.create_hamiltonian()
    psi0 = system.create_initial_state('1', '+')

    # Collapse operators: allow leakage/decay, but NOT sz dephasing
    c_ops = system.create_collapse_operators(leaking=(gamma > 0),
                                             decay=(kappa > 0),
                                             dephasing=False)

    # Times
    t_max = 100
    times1 = np.linspace(0, t_echo, 100)
    times2 = np.linspace(t_echo, t_max, 200)
    times_full = np.concatenate([times1, times2])

    # Pi pulse
    X_pi = qt.tensor(qt.qeye(system.N), qt.sigmax())

    def evolve_with_static_noise(delta_phi):
        # Add quasi-static noise as z-shift
        H_noise = H + 0.5 * delta_phi * qt.tensor(qt.qeye(system.N), qt.sigmaz())

        result1 = qt.mesolve(H_noise, psi0, times1, c_ops, [], options=qt.Options(store_states=True))
        rho_mid = result1.states[-1]
        rho_mid_pulsed = X_pi * rho_mid * X_pi.dag()
        result2 = qt.mesolve(H_noise, rho_mid_pulsed, times2, c_ops, [], options=qt.Options(store_states=True))

        return result1.states + result2.states

    # Run multiple realizations
    all_states = []
    for _ in range(num_realizations):
        delta_phi = np.random.normal(0, sigma)
        states = evolve_with_static_noise(delta_phi)
        all_states.append(states)

    # Average states
    avg_states = []
    for i in range(len(times_full)):
        rho_avg = sum([all_states[r][i] for r in range(num_realizations)]) / num_realizations
        avg_states.append(rho_avg)

    # Compute observables
    n_atom = [qt.expect(system.sm.dag() * system.sm, rho) for rho in avg_states]
    n_cavity = [qt.expect(system.a.dag() * system.a, rho) for rho in avg_states]
    coh = [abs(qt.expect(system.coh_ge, rho)) for rho in avg_states]

    # Plot
    plt.figure(figsize=(8, 5))
    plt.plot(times_full, n_atom, label="⟨n_atom⟩")
    plt.plot(times_full, n_cavity, label="⟨n_cavity⟩")
    plt.plot(times_full, coh, label="|coherence atom|")
    plt.legend()
    plt.xlabel("Time")
    plt.ylabel("Expectation value")
    plt.title("Spin Echo with Quasi-Static Noise")
    plt.grid(True)
    plt.show()


# Interactive sliders
interact(run_simulation,
         gamma=FloatSlider(min=0, max=0.5, step=0.01, value=0.0, description="Leaking γ"),
         kappa=FloatSlider(min=0, max=0.5, step=0.01, value=0.0, description="Decay κ"),
         sigma=FloatSlider(min=0.0, max=1.0, step=0.05, value=0.2, description="Noise σ"),
         t_echo=FloatSlider(min=0.1, max=60, step=0.5, value=30, description="Echo time"));


interactive(children=(FloatSlider(value=0.0, description='Leaking γ', max=0.5, step=0.01), FloatSlider(value=0…

# Visualization of the atom state on the Bloch sphere

## Visualization of the |+⟩ State 

We initialize the system in `|0⟩_cavity ⊗ |+⟩_atom` with **no coupling and no dissipation**.  
The atomic Hamiltonian causes a **rotation of the Bloch vector around the z-axis**. The vector remains in the XY-plane with **full length**.  

The main goal is to **visualize this rotation on the Bloch sphere**.  
The GIF shows the continuous precession of the Bloch vector in the XY-plane, demonstrating the coherent evolution of the |+⟩ state in the absence of interactions or noise.

(uncomment the last line to create the GIF)

In [12]:
system = AtomCavitySystem(N=10, g=0, w0=(1.0 * 2 * np.pi),
                              gamma=0, kappa=0, beta=0)

# Build Hamiltonian
H = system.create_hamiltonian()

# Initial state |0⟩_cavity ⊗ |+⟩_atom
psi11 = system.create_initial_state('0', '+')

# Collapse operators
c_ops = system.create_collapse_operators(leaking=False, decay=False, dephasing=False)

# Expectation values
e_ops = system.create_expectation_values(
    number_atom=True,
    number_cavity=True,
    coherence_ge=True,
    coherence_eg=True
)

# Time evolution
times = np.linspace(0, 20, 500)
result = qt.mesolve(H, psi11, times, c_ops, e_ops, options=qt.Options(store_states=True))

#system.create_bloch_gif(result, "plus_state.gif")

## Bloch Sphere Evolution with Dephasing

The system is initialized in `|0⟩_cavity ⊗ |+⟩_atom` as before, but now we include **atomic dephasing** (`β > 0`).  

- The Hamiltonian still causes **rotation around the z-axis**, but the dephasing gradually **reduces the length of the Bloch vector** in the XY-plane.  
- This corresponds to a **decay of coherence** over time while the populations remain unchanged.  

The GIF shows the **shrinking of the Bloch vector** in the XY-plane, illustrating the effect of decoherence on the atomic superposition state.

(uncomment the last line to create the GIF)

In [13]:
system = AtomCavitySystem(N=10, g=0, w0=(1.0 * 2 * np.pi),
                              gamma=0, kappa=0, beta=0.1)

# Build Hamiltonian
H = system.create_hamiltonian()

# Initial state |0⟩_cavity ⊗ |+⟩_atom
psi11 = system.create_initial_state('0', '+')

# Collapse operators
c_ops = system.create_collapse_operators(leaking=False, decay=False, dephasing=True)

# Expectation values
e_ops = system.create_expectation_values(
    number_atom=True,
    number_cavity=True,
    coherence_ge=True,
    coherence_eg=True
)

# Time evolution
times = np.linspace(0, 20, 500)
result = qt.mesolve(H, psi11, times, c_ops, e_ops, options=qt.Options(store_states=True))

#system.create_bloch_gif(result, "plus_state_dec.gif")

## Spin Echo with Quasi-Static Noise

Key points of this setup:

- The system starts in the `|0⟩_cavity ⊗ |+⟩_atom` state, creating **coherence in the XY-plane** of the Bloch sphere.  
- Instead of Lindblad dephasing, **quasi-static noise** is applied as a **random σ_z rotation** for each realization:
  \[
  H_\text{noise} = H + \frac{\delta\phi}{2}\, \sigma_z
  \]
  This represents a fixed, slow detuning of the atomic transition for each realization.
- A **π-pulse** is applied at `t_echo` to implement the **spin-echo sequence**, which reverses the effect of the quasi-static noise, allowing the Bloch vector to partially **refocus**.
- Running multiple realizations and averaging shows **decay of coherence before the echo**, then **partial recovery of coherence** after the π-pulse.

The GIF will visualize the **XY-plane rotation with noise**, and the echo will appear as a **re-lengthening of the Bloch vector** after `t_echo`.

(uncomment the last line to create the GIF)


In [14]:
# Parameters 

N = 10
g = 0
w0 = 1.0 * 2 * np.pi
gamma = 0.0
kappa = 0.0
beta = 0.0   # do quasi-static noise instead of Lindblad sz!
t_max = 60
t_echo = 10.25
num_realizations = 50
sigma = 0.2   # quasi-static dephasing strength

times1 = np.linspace(0, t_echo, 100)
times2 = np.linspace(t_echo, t_max, 400)
times_full = np.concatenate([times1, times2])

# Pi pulse on atom
X_pi = qt.tensor(qt.qeye(N), qt.sigmax())


# System setup
system = AtomCavitySystem(N=N, g=g, w0=w0, gamma=gamma, kappa=kappa, beta=beta)
H = system.create_hamiltonian()
psi0 = system.create_initial_state('1', '+')

# Expectation values for Bloch vector
e_ops = [system.coh_ge, system.coh_eg]


# Evolve with quasi-static noise
def evolve_with_static_noise(H, psi0, times1, times2, delta_phi):
    H_noise = H + 0.5 * delta_phi * qt.tensor(qt.qeye(N), qt.sigmaz())

    result1 = qt.mesolve(H_noise, psi0, times1, [], e_ops, options=qt.Options(store_states=True))
    rho_mid = result1.states[-1]
    rho_mid_pulsed = X_pi * rho_mid * X_pi.dag()
    result2 = qt.mesolve(H_noise, rho_mid_pulsed, times2, [], e_ops, options=qt.Options(store_states=True))

    result2.states = result1.states + result2.states
    result2.times = np.concatenate([times1, times2])
    return result2

# Run multiple realizations
all_states = []
for _ in range(num_realizations):
    delta_phi = np.random.normal(0, sigma)
    result = evolve_with_static_noise(H, psi0, times1, times2, delta_phi)
    all_states.append(result.states)

# Average over realizations
avg_states = []
for i in range(len(times_full)):
    rho_avg = sum([all_states[r][i] for r in range(num_realizations)]) / num_realizations
    avg_states.append(rho_avg)


# Create GIF
#system.create_bloch_gif(type('Result', (), {'states': avg_states, 'times': times_full})(), 
                        #"plus_state_spin_echo.gif")


## Coupling multiple Atoms to the cavity

## Visualization of the Atom states on multiple Blochspheres

Key points:

- `MultipleAtomsCavitySystem` is used with `num_atoms=3`, meaning three two-level atoms interact with the cavity.  
- Coupling `g=0.5` is **nonzero**, so now the atoms and cavity can exchange excitations.  
- The initial state is `|0⟩_cavity ⊗ |g,g,g⟩_atoms`, so all atoms start in the ground state.  
- No dissipation is included (`leaking=False, decay=False, dephasing=False`), so evolution is **fully coherent**.  
- The `create_multi_bloch_gif` function generates a **GIF showing the collective Bloch vector evolution** for all atoms, allowing visualization of **collective dynamics and correlations**.

This setup allows you to explore **collective phenomena** such as sub-/superradiance and entanglement in the cavity.

(uncomment the last line to create the GIF)



In [15]:
system = MultipleAtomsCavitySystem(N=5, g=0.5, w0=2*np.pi, num_atoms=3)
system.create_operators()

H = system.create_hamiltonian()
psi0 = system.create_initial_state('3', ['g', 'g', 'g'])
c_ops = system.create_collapse_operators(leaking=False, decay=False, dephasing=False)

times = np.linspace(0, 10, 150)
result = qt.mesolve(H, psi0, times, c_ops, [])

#system.create_multi_bloch_gif(result, "two_atoms.gif")
